<a href="https://colab.research.google.com/github/hema082004/Fake_news/blob/main/Telugu_Fake_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
# Clone the GitHub repository (if it is public)
!git clone https://github.com/prab786/Multi-Dimensional-Telugu-Fake-Newsdataset.git

# Go inside the project folder
%cd Multi-Dimensional-Telugu-Fake-Newsdataset


Cloning into 'Multi-Dimensional-Telugu-Fake-Newsdataset'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 26 (delta 9), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 492.66 KiB | 7.04 MiB/s, done.
Resolving deltas: 100% (9/9), done.
/content/Multi-Dimensional-Telugu-Fake-Newsdataset/Multi-Dimensional-Telugu-Fake-Newsdataset


In [25]:
!pip install numpy pandas scikit-learn matplotlib seaborn nltk joblib


In [26]:
import pandas as pd
import json

# Load the dataset.json file
with open('dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)
df.head()

,serial no,site,claim,image,date,fact
0,0,https://factly.in/telugu-digitally-edited-phot...,శ్రీ రాముడి యొక్క 3D చిత్రం దుబాయ్‌లోని బుర్జ...,https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",బుర్జ్ ఖలీఫా పైన శ్రీ రాముని చిత్రాన్ని ప్రదర్...
1,1,https://factly.in/telugu-this-way2news-news-ar...,"చెప్పులు వేసుకొని అయోధ్య ఆలయంలోకి ప్రవేశించి, ...",https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",ఈ వార్తను ‘Way2News’ ప్రచురించలేదు. ఇది వారి ...
2,2,https://factly.in/telugu-an-unrelated-video-of...,అయోధ్యలో జరగనున్న రామమందిర ప్రతిష్టాపన వేడుకక...,https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",వైరల్ వీడియోని జార్జియా మెలోని తన X (ట్విట్ట...
3,3,https://factly.in/telugu-the-display-of-lord-r...,శ్రీనగర్‌లోని క్లాక్ టవర్‌పై శ్రీరాముడి చిత్ర...,https://factly.in/wp-content/uploads//2024/01/...,"January 22, 2024",ఈ వీడియోలో కనిపిస్తున్న శ్రీరాముడి చిత్ర ప్రద...
4,4,https://factly.in/telugu-a-video-of-nihal-sing...,శ్రీరాముని విగ్రహ ప్రతిష్టాపన వేడుకకు రెండు చ...,https://factly.in/wp-content/uploads//2024/01/...,"January 22, 2024",వైరల్ వీడియోలో కనిపిస్తున్న వ్యక్తి బీహార్‌కు...


In [27]:
# Load annotation labels
label_df = pd.read_csv('label.csv')

# Merge on serial number (common key for both)
df = df.merge(label_df, left_on="serial no", right_on="ID", how="inner")
df.head()

,serial no,site,claim,image,date,fact,ID,What is the rating of the news?,Any visual data required to judge the verdict?,To what extent is the news harmful to society?,Category of harm?
0,0,https://factly.in/telugu-digitally-edited-phot...,శ్రీ రాముడి యొక్క 3D చిత్రం దుబాయ్‌లోని బుర్జ...,https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",బుర్జ్ ఖలీఫా పైన శ్రీ రాముని చిత్రాన్ని ప్రదర్...,0,FALSE,Textual data is enough,1,Harmless
1,1,https://factly.in/telugu-this-way2news-news-ar...,"చెప్పులు వేసుకొని అయోధ్య ఆలయంలోకి ప్రవేశించి, ...",https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",ఈ వార్తను ‘Way2News’ ప్రచురించలేదు. ఇది వారి ...,1,FALSE,Visual data required,2,Threat to honor and trust in people and companies
2,2,https://factly.in/telugu-an-unrelated-video-of...,అయోధ్యలో జరగనున్న రామమందిర ప్రతిష్టాపన వేడుకక...,https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",వైరల్ వీడియోని జార్జియా మెలోని తన X (ట్విట్ట...,2,FALSE,Visual data required,1,Harmless
3,3,https://factly.in/telugu-the-display-of-lord-r...,శ్రీనగర్‌లోని క్లాక్ టవర్‌పై శ్రీరాముడి చిత్ర...,https://factly.in/wp-content/uploads//2024/01/...,"January 22, 2024",ఈ వీడియోలో కనిపిస్తున్న శ్రీరాముడి చిత్ర ప్రద...,3,half true & half false,Visual data required,1,Harmless
4,4,https://factly.in/telugu-a-video-of-nihal-sing...,శ్రీరాముని విగ్రహ ప్రతిష్టాపన వేడుకకు రెండు చ...,https://factly.in/wp-content/uploads//2024/01/...,"January 22, 2024",వైరల్ వీడియోలో కనిపిస్తున్న వ్యక్తి బీహార్‌కు...,4,FALSE,Visual data required,1,Harmless


In [28]:
df = df.rename(columns={
    "claim": "Text",
    "Q1": "Label"   # Q1 = True / False rating label
})


In [29]:
# Rename 'claim' to 'Text' if it exists
if 'claim' in df.columns:
    df = df.rename(columns={"claim": "Text"})
elif 'Text' not in df.columns:
    raise KeyError("Column 'claim' or 'Text' not found in DataFrame. Please ensure 'claim' exists or has already been renamed to 'Text'.")

# Rename 'What is the rating of the news?' to 'Label' if it exists
if 'What is the rating of the news?' in df.columns:
    df = df.rename(columns={"What is the rating of the news?": "Label"})
elif 'Label' not in df.columns:
    raise KeyError("Column 'What is the rating of the news?' or 'Label' not found in DataFrame. Please ensure the rating column exists or has already been renamed to 'Label'.")

# Convert string labels to numerical (0 for FALSE, 1 for others)
df['Label'] = df['Label'].apply(lambda x: 0 if x == 'FALSE' else 1)

In [30]:
print(df.shape)
print(df['Label'].value_counts())
df.sample(5)


(510, 11)
Label
0    369
1    141
Name: count, dtype: int64


,serial no,site,Text,image,date,fact,ID,Label,Any visual data required to judge the verdict?,To what extent is the news harmful to society?,Category of harm?
343,343,https://factly.in/telugu-the-2018-scedule-of-t...,: భారత ఎన్నికల సంఘం తెలంగాణ 2023 అసెంబ్లీ ఎన్న...,https://factly.in/wp-content/uploads//2023/07/...,"July 20, 2023",2023 తెలంగాణ అసెంబ్లీ ఎన్నికలకు సంబంధించిన అధి...,343,0,Textual data is enough,1,Harmless
286,286,https://factly.in/telugu-a-video-of-a-nomadic-...,ఈ వీడియో మయన్మార్ నుండి భారతదేశంలోకి అక్రమంగా...,https://factly.in/wp-content/uploads//2023/08/...,"August 14, 2023",ఈ దృశ్యాలు ఇరాన్‌లోని ఒక సంచార కుటుంబాన్ని చూ...,286,0,Visual data required,2,Confusion and anxiety about society
302,302,https://factly.in/telugu-bhumana-karunakar-red...,భూమన కరుణాకర్ రెడ్డి తానే స్వయంగా ఎన్నికల అఫి...,https://factly.in/wp-content/uploads//2023/08/...,"August 8, 2023",భూమన కరుణాకర్ రెడ్డి ఎన్నికల అఫిడవిట్‌లో తాను...,302,0,Textual data is enough,1,Threat to honor and trust in people and companies
481,481,https://factly.in/telugu-morphed-photo-shared-...,ధర్మం చేయకుంటే ఈ సారి కూడా కేసీఆర్‌కు వోటెస్త...,https://factly.in/wp-content/uploads//2023/05/...,"May 8, 2023",పోస్టులో షేర్ చేసిన ఫోటో మార్ఫ్ చేయబడినది. అస...,481,0,Visual data required,1,Threat to correct understanding of politics an...
295,295,https://factly.in/telugu-an-old-video-from-bih...,ఒక హిందూ మహిళపై కొందరు ముస్లింలు దాడి చేయడాన్...,https://factly.in/wp-content/uploads//2023/08/...,"August 10, 2023",ఈ వీడియో 2021లో బీహార్‌లో జరిగిన ఒక సంఘటనకు చ...,295,0,Visual data required,2,Prejudice against country and race


In [31]:
import pandas as pd
import json

# Load dataset.json
with open('dataset.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.DataFrame(data)

# Load labels and merge
label_df = pd.read_csv('label.csv')
df = df.merge(label_df, left_on="serial no", right_on="ID", how="inner")

# Rename to ML format
df = df.rename(columns={"claim": "Text", "What is the rating of the news?": "Label"})

# Convert 0 = Fake, 1/2 = Real
df['Label'] = df['Label'].apply(lambda x: 0 if x == 'FALSE' else 1)

print("Dataset Loaded Successfully")
print("Shape:", df.shape)
print(df['Label'].value_counts())
df.head()

Dataset Loaded Successfully
Shape: (510, 11)
Label
0    369
1    141
Name: count, dtype: int64


,serial no,site,Text,image,date,fact,ID,Label,Any visual data required to judge the verdict?,To what extent is the news harmful to society?,Category of harm?
0,0,https://factly.in/telugu-digitally-edited-phot...,శ్రీ రాముడి యొక్క 3D చిత్రం దుబాయ్‌లోని బుర్జ...,https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",బుర్జ్ ఖలీఫా పైన శ్రీ రాముని చిత్రాన్ని ప్రదర్...,0,0,Textual data is enough,1,Harmless
1,1,https://factly.in/telugu-this-way2news-news-ar...,"చెప్పులు వేసుకొని అయోధ్య ఆలయంలోకి ప్రవేశించి, ...",https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",ఈ వార్తను ‘Way2News’ ప్రచురించలేదు. ఇది వారి ...,1,0,Visual data required,2,Threat to honor and trust in people and companies
2,2,https://factly.in/telugu-an-unrelated-video-of...,అయోధ్యలో జరగనున్న రామమందిర ప్రతిష్టాపన వేడుకక...,https://factly.in/wp-content/uploads//2024/01/...,"January 23, 2024",వైరల్ వీడియోని జార్జియా మెలోని తన X (ట్విట్ట...,2,0,Visual data required,1,Harmless
3,3,https://factly.in/telugu-the-display-of-lord-r...,శ్రీనగర్‌లోని క్లాక్ టవర్‌పై శ్రీరాముడి చిత్ర...,https://factly.in/wp-content/uploads//2024/01/...,"January 22, 2024",ఈ వీడియోలో కనిపిస్తున్న శ్రీరాముడి చిత్ర ప్రద...,3,1,Visual data required,1,Harmless
4,4,https://factly.in/telugu-a-video-of-nihal-sing...,శ్రీరాముని విగ్రహ ప్రతిష్టాపన వేడుకకు రెండు చ...,https://factly.in/wp-content/uploads//2024/01/...,"January 22, 2024",వైరల్ వీడియోలో కనిపిస్తున్న వ్యక్తి బీహార్‌కు...,4,0,Visual data required,1,Harmless
